In [ ]:
import sys
sys.path.append('../')
import psycopg2
from configparser import ConfigParser
import requests
from datetime import datetime
from config.settings import settings

### Connect to the database

In [49]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

In [52]:
def load_config(filename='../database/database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config


In [53]:
config = load_config()
connect(config)

Connected to the PostgreSQL server.


<connection object at 0x0000026E528EBAB0; dsn: 'user=postgres password=xxx dbname=debris_flow_dt host=localhost', closed: 0>

### Wrap data from web

In [44]:
STATION_ID = "34362"
API_URL = f"https://stationsweb.awekas.at/api/station/data?id={STATION_ID}"

def fetch_station_data():
    r = requests.get(API_URL, timeout=15)
    r.raise_for_status()
    return r.json()

In [45]:
def parse_record(json_data):
    ts = datetime.strptime(json_data["data"]["time"], "%Y-%m-%d %H:%M")

    rainfall_mm = float(json_data["data"].get("rain", 0.0))
    temperature = float(json_data["data"].get("temp", None))
    humidity = float(json_data["data"].get("humidity", None))

    lat = json_data["station"]["lat"]
    lon = json_data["station"]["lon"]

    return {
        "station_id": STATION_ID,
        "timestamp": ts,
        "rainfall_mm": rainfall_mm,
        "temperature_c": temperature,
        "humidity_percent": humidity,
        "lat": lat,
        "lon": lon
    }


In [46]:
def calculate_intensity(current_mm, delta_minutes):
    if delta_minutes == 0:
        return 0
    return (current_mm / delta_minutes) * 60


In [ ]:
def insert_rainfall_data(record, intensity):
    sql = """
    INSERT INTO rainfall_data (
        station_id, timestamp, rainfall_mm, intensity_mm_h,
        temperature_c, humidity_percent, geom
    )
    VALUES (
        %s, %s, %s, %s, %s, %s,
        ST_SetSRID(ST_MakePoint(%s, %s), 4326)
    )
    ON CONFLICT DO NOTHING;
    """

    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute(sql, (
        record["station_id"],
        record["timestamp"],
        record["rainfall_mm"],
        intensity,
        record["temperature_c"],
        record["humidity_percent"],
        record["lon"],
        record["lat"]
    ))
    conn.commit()
    cur.close()
    conn.close()
